In [2]:
import cv2
import mediapipe as mp

### 얼굴 메시 추출
- 468개의 3D 얼굴 랜드마크를 추정

In [3]:
# 추정 결과를 시각화하는 라이브러리
mp_drawing = mp.solutions.drawing_utils

# 얼굴 메시 추출 라이브러리 가져오기
mp_face_mesh = mp.solutions.face_mesh

# 시각화 설정
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# 영상 불러오기
cap = cv2.VideoCapture('face.mp4')

# 얼굴 메시 추출
with mp_face_mesh.FaceMesh( min_detection_confidence=0.5,  # 최소 탐지 신뢰도
                           # 최소 추적 신뢰도
                          min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret :
            break
            
        frame.flags.writeable = False
        
        # 메시 검출 (랜드마크 검출)
        results = face_mesh.process(frame)
        
        frame.flags.writeable = True
        
        # 랜드마크를 서로 연결
        # 랜드마크 점이 검출되었다면
        if results.multi_face_landmarks:
            # 검출 랜드마크를 하나씩 가져오기
            for face_landmark in results.multi_face_landmarks:
                # 랜드마크끼리 선으로 연결
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmark,
                    # 얼굴 전체
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    # 그리기 설정
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec
                )
            
        cv2.imshow("face mesh", frame)
        
        key = cv2.waitKey(33)
        
        if key == 49:
            break
cap.release()
cv2.destroyAllWindows()

### 얼굴, 왼손, 오른손, 동작 검출(스켈레톤)

In [9]:
# 그리기와 스켈레톤 추출기 가져오기
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture('face2.mp4')

drawing_spec1 = mp_drawing.DrawingSpec(thickness = 3, color = (0,0,255))
drawing_spec2 = mp_drawing.DrawingSpec(thickness = 3, color = (255,0,0))

with mp_holistic.Holistic(min_detection_confidence=0.5,
                         min_tracking_confidence=0.5) as holistic :
    # 비디오가 열렸다면
    while cap.isOpened() :
        ret, image = cap.read()
        
        if not ret :
            print('비디오 읽기 실패')
            cap.release()
            cv2.destroyAllWindows()
            break
            
        image.flags.writeable = False
        results = holistic.process(image)
        image.flags.writeable = True
        
        # 랜드마크와 랜드마크를 연결하는 선 그리기
        mp_drawing.draw_landmarks(image, results.pose_landmarks,
                                   mp_holistic.POSE_CONNECTIONS,
                                   landmark_drawing_spec=drawing_spec1,
                                   connection_drawing_spec=drawing_spec2)    
        
        
        cv2.imshow("video holistic", image)
        
        key = cv2.waitKey(33)
        
        if key == 49:
            break
            
cap.release()
cv2.destroyAllWindows()